# DLA functions

In [1]:
import pandas as pd

import _clean_data
from shared_utils import geography_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
'''
df = _clean_data.make_clean_data()
df.to_parquet("./data.parquet")
'''

df = pd.read_parquet("./data.parquet")

## Aggregation

* Aggregate by some group, get count
* Also sort in descending order

In [3]:
# How to use the geography_utils function to return 1 df, no merging needed
dist_years1 = geography_utils.aggregate_by_geography(
    df, 
    group_cols = ["dist"],
    sum_cols = ["total_requested"],
    count_cols = ["project_no"],
    nunique_cols = ["primary_agency_name", "prefix"],
)

dist_years1.head()
#dist_agencyn = (df >> group_by(_.dist) >> summarize(n=_.primary_agency_name.nunique()) >> arrange(-_.n))
#dist_prefixn = (df >> group_by(_.dist) >> summarize(n=_.prefix.nunique()) >> arrange(-_.n))
#sum_funds = df>>group_by(_.dist)>>summarize(n=_.total_requested.sum()) >> arrange(-_.n)


,dist,total_requested,project_no,prefix,primary_agency_name
0,1,1.751416e+08,1161,25,20
1,3,1.561961e+09,2517,98,62
2,4,2.820382e+09,3248,110,151
3,6,1.597301e+09,3084,71,55
4,7,4.264832e+09,2787,107,108


In [4]:
dist_years2 = geography_utils.aggregate_by_geography(
    df, 
    group_cols = ["prepared_y", "dist"],
    sum_cols = ["total_requested"],
    count_cols = ["project_no"],
    nunique_cols = ["primary_agency_name", "prefix"]
).sort_values(["prepared_y", "dist"], ascending=[False, True])

dist_years2.head()

,prepared_y,dist,total_requested,project_no,prefix,primary_agency_name
120,2021.0,0,-1.140550e+03,1.0,1.0,1.0
106,2021.0,1,1.217239e+07,96.0,13.0,9.0
107,2021.0,2,7.705425e+06,75.0,15.0,13.0
108,2021.0,3,2.378141e+08,248.0,34.0,45.0
109,2021.0,4,1.602708e+08,336.0,37.0,91.0


### Take aggregation function a step further

If you're always producing 2 basic processed, aggregated datasets by district, county, MPO, etc, then this is one way to get both at once.

In [5]:
def aggregate_datasets(df, aggregate_by=["dist"]):
    """
    Parameters:
    df: pandas.DataFrame 
        Cleaned data, the result of _clean_data functions.
    aggregate_by: list.
        List of functions to group by
        Ex: district, county, MPO, RTPA
    
    Returns: two pandas.DataFrames.
            Metrics calculated: 
            sum of total requested, # unique agencies, # unique prefixes
            
            first df: aggregated to group (each row is district)
            second df: aggregated to year-group (each row is district-year)
    """
    
    by_geography = (
        geography_utils.aggregate_by_geography(
            df[df[aggregate_by].notna()], 
            group_cols = [aggregate_by],
            sum_cols = ["total_requested"],
            nunique_cols = ["primary_agency_name", "prefix"]
            ).sort_values(["primary_agency_name"], ascending=False)
    )
    
    by_geography_year = (
        geography_utils.aggregate_by_geography(
            df[(df[aggregate_by].notna()) & (df.prepared_y.notna())], 
            group_cols = ["prepared_y", aggregate_by],
            sum_cols = ["total_requested"],
            nunique_cols = ["primary_agency_name", "prefix"]
        ).sort_values(["prepared_y", aggregate_by], ascending=[False, True])
    )
        
    return by_geography, by_geography_year
  
    
# Do it all at once
# df = _clean_data.make_clean_data()
# OR, df = catalog.CLEANED_UP_DATASET.read()
df = pd.read_parquet("./data.parquet")


dist_years1, dist_years2 = aggregate_datasets(df, aggregate_by="dist")
mpo_years1, mpo_years2 = aggregate_datasets(df, aggregate_by="mpo")
agency_years1, agency_years2 = aggregate_datasets(df, aggregate_by="agency")

## Chart Functions

In [6]:
import altair as alt

from siuba import *

from shared_utils import styleguide
from shared_utils import calitp_color_palette as cp

alt.themes.register("calitp_theme", styleguide.calitp_theme)
# enable
alt.themes.enable("calitp_theme")

ThemeRegistry.enable('calitp_theme')

In [7]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Raleway');
@import url('https://fonts.googleapis.com/css?family=Nunito+Sans');
@import url('https://fonts.googleapis.com/css?family=Bitter');
</style>

With `altair`, you can reuse bits of the code to build a chart.

If there's a "base", some piece that is used over and over for a bar chart or a line chart, you can put that piece in its own function. Future functions can invoke that function, then add additional components, similar to how `.properties()`, `.configure_title()`, etc are added.

In [8]:
# Rename columns and then wrangle further for labeling with a function 
dist_years1 = dist_years1.rename(columns = {
    "primary_agency_name": "n_agencies",
    "prefix": "unique_prefixes",
})

# Or, use a dictionary to store all the various labeling needed
# Can use this to address other labeling, esp for random places
# Or, put this dictionary within 
LABEL_DICT = {
    "prepared_y": "Year",
    "dist": "District",
    "total_requested": "Total Requested",
}

# Then, use a function to clean up labels 
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word


In [9]:
print(labeling("mpo"))
print(labeling("dist"))
print(labeling("total_requested"))

MPO
District
Total Requested


In [10]:
def base_bar(df, x_col, y_col):
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(f"{x_col}:N", title=labeling(x_col)),
                 # Pass the labeling function here. Columns need to be renamed to use this.
                 y=alt.Y(f"{y_col}:Q", title=labeling(y_col)),
             )
            )
    
    return chart

# Ehhh, this might be too complicated...
# The stacked and grouped bar charts may be overkill...
# Use the base bar function, then add further encodings and properties
def stacked_bar_chart(df, x_col, y_col, color_col, chart_title):
    base = base_bar(df, x_col, y_col)
    
    chart = (base.encode(
                 color=alt.Color(f"{color_col}:N", 
                                 scale=alt.Scale(
                                     range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
                                 legend=alt.Legend(
                                     title=labeling(color_col))
                                )
             ).properties(title=chart_title)
    )
    
    return chart


def grouped_bar_chart(df, x_col, y_col, color_col, column_col, chart_title):
    base = stacked_bar_chart(df, x_col, y_col, color_col, chart_title)
    
    chart = (base.encode(
                 column=alt.Column(f"{column_col}:N")
             ).properties(title=chart_title)
    )
    
    return chart

In [11]:
base_bar(dist_years1, x_col="dist", 
         y_col="unique_prefixes").properties(
    title="Number of Unique Prefixes by District")

alt.Chart(...)

In [12]:
base_bar(dist_years2, x_col="prepared_y", 
          y_col="primary_agency_name").properties(
    title="New Chart Title"
)

alt.Chart(...)

In [13]:
base_bar(mpo_years1, x_col="mpo", 
         y_col="prefix").properties(
    title="Number of Unique Prefixes by MPO")

alt.Chart(...)

In [14]:
stacked_bar_chart(dist_years2, x_col="prepared_y", 
          y_col="primary_agency_name", color_col="dist",
          chart_title="Number of Obligations by District")

alt.Chart(...)

In [15]:
grouped_bar_chart(dist_years2[dist_years2.prepared_y >= 2018], 
                  x_col="dist", y_col="prefix", color_col="dist",
                  column_col="prepared_y", chart_title="Title")

alt.Chart(...)

In [16]:
base = base_bar(dist_years2, "dist", "primary_agency_name")
# Doing a grouped bar chart might be sufficient this way
base.encode(
    column = alt.Column("prepared_y:O"),
    color=alt.Color("dist:N")
)

alt.Chart(...)

## Interactive Widgets

In [17]:
from plotnine import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

In [18]:
def summarize_and_plot(df, select_col, place):
    subset = df[df[select_col]==place].rename(
        columns = {
            "fed_requested": "Federal",
            "ac_requested": "AC",
            "total_requested": "Total",
        }
    )
    
    prefix_count_n = subset >> count(_.prefix)

    display(Markdown(f"**Summary Statistics for {place}**"))
    display(Markdown(f"The number of obligations {place} has is {len(prefix_count_n)}"))
    
    display(Markdown(
        f"The number of prefix codes {place} uses is {subset.prefix.nunique()}"))

    pd.set_option("display.max_columns", None)

    funds = subset[['Federal','AC','Total']].describe()
    display(funds.style.format(precision=2, na_rep='MISSING', thousands=","))

    display(Markdown(f"**Top Project Types in {place}**"))

    work_df = subset >> count(_.type_of_work) >> arrange(-_.n)
    display(work_df.head(5))
    
    ax1 = (prefix_count_n
            >> ggplot(aes("prefix", "n", fill="prefix")) 
               + geom_col() 
               + theme(axis_text_x = element_text(angle = 45 , hjust=1))
               + labs(title='Agency Program Codes', x='Program Codes', 
                      y='Number of Obligations', fill="Program Type")
        )
    display(ax1)


In [19]:
select_col = "agency"

dropdown = widgets.Dropdown(
    description=f"{select_col.title()}",
    options=df[select_col].sort_values().unique().tolist(),
)
output = widgets.Output()

display(dropdown)
display(output)

def on_selection(*args):
    output.clear_output()
    with output:
        summarize_and_plot(df, select_col, dropdown.value)

dropdown.observe(on_selection, names="value")
on_selection()

Dropdown(description='Agency', options=('Access Services', 'Agoura Hills', 'Ala-Con Costa T', 'Alameda', 'Alam…

Output()

### Generalize interactive function

In [20]:
## Think about how to wrap above interactive widget into a function
# Be able to select agency, county, district, etc
def interactive_widget(df, select_col):

    dropdown = widgets.Dropdown(
        description=f"{select_col.title()}",
        options=df[select_col].sort_values().unique().tolist(),
    )
    output = widgets.Output()

    display(dropdown)
    display(output)

    def on_selection(*args):
        output.clear_output()
        with output:
            summarize_and_plot(df, select_col, dropdown.value)

    dropdown.observe(on_selection, names="value")
    on_selection()

In [21]:
interactive_widget(df, "agency")

Dropdown(description='Agency', options=('Access Services', 'Agoura Hills', 'Ala-Con Costa T', 'Alameda', 'Alam…

Output()

In [22]:
interactive_widget(df, "mpo")

Dropdown(description='Mpo', options=('AMBAG', 'BCAG', 'CFCG', 'COFCG', 'ER NONE', 'FCOG', 'FSTIP', 'KCAG', 'KC…

Output()

## Merge with geography

In [23]:
import geopandas as gpd
import intake
import pandas as pd

from shared_utils import geography_utils 

catalog = intake.open_catalog("*.yml")

def merge_shapefile(df, geography):
    GEOG_DICT = {
        # key: column name for geography
        # value: name in catalog.yml
        "county": "county_boundary",
        "dist": "district_boundary", 
        "mpo": "mpo_boundary",
    }
    
    # This returns the catalog entry name
    catalog_entry = GEOG_DICT[geography]
    
    # Merge in shapefile from catalog
    gdf = catalog[catalog_entry].read().to_crs(geography_utils.WGS84)
    
    # Prior to merging, make sure catalog entries have same column names 
    # otherwise, merging won't work
    RENAME_COLS = {
        # key: name of column in open data
        # value: new name for column to match df
        "COUNTY": "county",
        "MPO": "mpo",
    }
    
    gdf = pd.merge(gdf.rename(columns = RENAME_COLS), 
                   df, 
                   # geography is defined earlier on in function
                   # ex: dist, mpo, county. 
                   # it's the name of the column in the cleaned df
                   on = geography,
                   # left merge to keep all polygons
                   how = "left",
                   # is it a 1:m join? good to check. do polygons overlap?
                   validate = "1:m"
                  )
    
    return gdf